In [ ]:
import os
os.chdir('/home/denisalpino/dev/FinABYSS') # change path to the current directory

import polars as pl

from parsers.yahoo_parser import YahooFinanceParser

##### **Initialization**

In [ ]:
START, END = "2023-09-01", "2025-03-18"
PROXIES = eval(os.getenv("PROXIES")) # type: ignore

yfp = YahooFinanceParser()

##### **Articles URLs parsing**

Collect all urls on articles due the whole period

In [ ]:
results = await yfp.get_all_news_in_range(START, END, retry=1)

Save results to the Parquet file

In [ ]:
(
    pl.Series(values=list(results), name="news_urls")
    .to_frame()
    .write_parquet("./data/news_urls.parquet")
)

##### **Articles parsing**

Read previously parsed URLs

In [ ]:
urls = pl.read_parquet('data/news_urls.parquet')

Add proxies to the parser instance

In [ ]:
for proxy in PROXIES:
    yfp.add_proxy(url=proxy)

Parse and save all artiucles data

In [ ]:
await yfp.get_all_articles(
    urls.get_column("news_urls").to_list(),
    file_path="data/articles.parquet",
    logging=True,
    use_proxies=True,
    tqdm_progress=True
)